In [1]:
import pandas as pd
import mwparserfromhell as hell
import unicodedata
import html 
import spacy
import time
import numpy as np
import string
nlp = spacy.load('en_core_web_trf')

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from tqdm.auto import tqdm

In [3]:
def get_headings(wikicode, check):
    wikicode = hell.parse(article)
    cutoff_list = ['Seealso',
                   'References',
                   'Footnotes',
                   'Notes',
                   'Bibliography',
                   'Publications',
                   'Footnotesandreferences',
                   'Primarysources',
                   'Citations',
                   'Notesandreferences',
                   'Externallinks',
                   'Furtherreading',
                   'Referencesandfurtherreading',
                   'References/sources',
                   'Importantpublications',
                   'Sources']
    bad_headings = []
    for heading in wikicode.filter_headings():
        for bad_heading in cutoff_list:
            if heading.replace('=','').replace(' ','') == bad_heading:
                bad_headings.append(heading)
    if check == True:
        # returns error if bad_headings == []
        # useful later to check for poor quality articles
        bad_heading = [bad_headings[0]]
    else:
        bad_heading = bad_headings[:1]
    headings = [str(j) for j in wikicode.filter_headings() if str(j).replace(' ','').replace('=','') not in cutoff_list] + bad_heading
    return headings

In [4]:
def get_templates(wikicode):
    key_templates = ['val|','math|','frac|','mvar|','convert|']
    templates = []
    for template in wikicode.filter_templates():
        new_template = str(template).replace(' ','').split('{{')[1].split('|')[0]+'|'
        templates.append(template) if new_template not in key_templates else 0
    return templates

In [5]:
def extract_math(section, just_equations):
    out = []
    for i in section.split('<math>')[1:]:
        math = '<math>'+i.split('</math>')[0]+'</math>'
        if just_equations == True:
            out.append(math) if '=' in math and math not in out else 0
        else:
            out.append(math) if math not in out else 0
    return out

In [6]:
def correct_math_tags(sentence):

    tags = []
    math = []
    for i in sentence.split('<math')[1:]:
        tag = i.split('>')[0]
        tags.append(tag) if tag != '' else 0

    for tag in tags:
        sentence = sentence.replace(tag,'')
    
    return sentence

In [7]:
def extract_sentences(section):
    # replace <math> terms with 'MATH_1_MATH' etc...

    math = extract_math(section, just_equations=False)
    math_swaps = [(math[j], 'M_%d_M'%(j)) for j in range(len(math))]

    for swap in math_swaps:
        section = section.replace(swap[0], swap[1]).replace('\n',' ')

    math_swaps.reverse()

    sentences = [str(sent) for sent in nlp(section).sents if str(sent).replace('\n','').replace(' ','') != '']

    new_sentences = []
    for sent in sentences:
        for swap in math_swaps:
            sent = sent.replace(swap[1], swap[0])
        new_sentences.append(sent)

    return new_sentences

In [8]:
def is_equation_broken(sentence):
    for i in range(len(sentence)):
        segment = sentence[:i+7]
        if ('</math>' in segment) and ('<math>' not in segment):
            value = True
            break
        else:
            value = False
    return value

In [9]:
def random_sentence(i, size):
    while True:
        idx = np.random.choice(len(structured_sentences))
        if idx != i:
            break
            
    rand_article = structured_sentences[idx]

    rand_section = rand_article[np.random.choice(len(rand_article))]

    if size == 1:
        rand_sentence = rand_section[np.random.choice(len(rand_section))]
    elif size == 2:
        k = np.random.choice(len(rand_section)) - 1
        rand_sentence = rand_section[k] + ' ' + rand_section[k+1]
        
    return rand_sentence

In [10]:
#df = pd.read_csv('WikiRaw.csv', encoding = 'UTF-8')
#df.head()
#old_articles = list(df.article)
#articles = list(df.article)

In [11]:
df = pd.read_pickle('wiki_math.pkl')
lists = df[df['Title'].str.contains('List of')]
df = pd.concat([df, lists, lists]).drop_duplicates(keep=False)

articles = list(df.Text)

In [12]:
def equations(article):
    eqs = []
    for i in article.split('<math>')[1:]:
        math = i.split('</math>')[0]
        if '=' in math:
            eqs.append(math)
    return list(dict.fromkeys(eqs))

In [13]:
math_articles = [i for i in articles if len(equations(i)) > 9]
len(math_articles)/len(articles)

0.18293718166383702

In [14]:
len(math_articles)

8620

In [17]:
articles = math_articles
df

,Title,Text,Categories
0,Albedo,{{Short description|Ratio of reflected radiati...,"['Land surface effects on climate', 'Climate c..."
1,Aristotle,{{short description|Classical Greek philosophe...,"['Aristotle', 'Aristotelianism *', '384 BC bir..."
2,Arithmetic mean,{{Short description|Numbers average}}\n{{broad...,['Means']
3,Analysis of variance,{{short description|Collection of statistical ...,"['Analysis of variance', 'Design of experiment..."
4,Affirming the consequent,{{Short description|type of fallacious argumen...,['Propositional fallacies']
...,...,...,...
47354,Wikipedia:Reference desk/Archives/Mathematics/...,<noinclude><!-- From archive header-->\n{{#ife...,"[""Pages with parser function time errors</stro..."
47355,Draft:SEPL,"<!-- Important, do not remove this line before...",[]
47356,Online fair division,'''Online fair division''' is a class of [[fai...,['Fair division']
47357,Draft:Quasi-steady-state approximation and par...,{{AfC submission|t||ts=20210319153825|u=Agnesp...,[]


In [18]:
while True:
    new_articles = []
    loop = tqdm(range(len(articles)))
    for i in loop:

        article = ' '.join([i for i in articles[i].split(' ') if i != '']).replace('. </math>','</math>.').replace('.</math>','</math>.')

        # filter comments
        while True:
            wikicode = hell.parse(article)
            for comment in wikicode.filter_comments():
                article = article.replace(str(comment), '')
            if len(wikicode.filter_comments()) == 0:
                #print('comments')
                break

        # filter arguments
        while True:
            wikicode = hell.parse(article)
            for argument in wikicode.filter_arguments():
                article = article.replace(str(argument), '')
            if len(wikicode.filter_arguments()) == 0:
                #print('arguments')
                break

        # filter templates ignoring mvar|, math|, frac|, val|, and convert|
        while True:
            wikicode = hell.parse(article)
            templates = get_templates(wikicode)
            for template in templates:
                article = article.replace(str(template), '')
            if len(templates) == 0:
                #print('templates')
                break

        # filter external_links
        while True:
            wikicode = hell.parse(article)
            for link in wikicode.filter_external_links():
                article = article.replace(str(link), '')
            if len(wikicode.filter_external_links()) == 0:
                #print('external_links')
                break

        # filter and replace html entities
        while True:
            wikicode = hell.parse(article)
            html_text = [str(j) for j in wikicode.filter_html_entities()]
            html_conv = [str(unicodedata.normalize('NFKD', html.unescape(j))) for j in html_text]
            for j in range(len(html_text)):
                article = article.replace(html_text[j], html_conv[j])
            if len(html_text) == 0:
                #print('html_text')
                break

        # filter and replace references
        while True:
            wikicode = hell.parse(article)
            refs = [str(j) for j in wikicode.filter_tags() if '<ref' in str(j)]
            for ref in refs:
                article = article.replace(ref,'')
            if len(refs) == 0:
                #print('refs')
                break

        # filter and replace ''' and '' tags (including those in math| etc.) 
        while True:
            wikicode = hell.parse(article)
            tags = list(dict.fromkeys([str(i) for i in wikicode.filter_tags()]))
            new_tags = []
            for tag in tags:
                if ('<math' not in tag) and ("''" in tag):
                    new_tag = tag.replace("'''",'').replace("''",'')
                    new_tags.append(new_tag)
                else:
                    new_tags.append(tag)
            if len(tags) > 0:
                for j in range(len(tags)):
                    article = article.replace(tags[j], new_tags[j])
            #count ''' and '' in new_tags
            count = 0
            for tag in new_tags:
                if '<math' not in tag:
                    count += tag.count("''") + tag.count("'''")
            if count == 0:
                #print("''' and ''")
                break

        # filter wikitable tags and class
        while True:
            wikicode = hell.parse(article)
            tags = [str(j) for j in wikicode.filter() if 'class=' in str(j).replace(' ','')]
            for tag in tags:
                article = article.replace(tag,'')
            if len(tags) == 0:
                #print('class=')
                break

        # filter [[File: tags
        while True:
            wikicode = hell.parse(article)
            tags = [str(j) for j in wikicode.filter() if '[file:' in str(j).replace(' ','').lower()]
            for tag in tags:
                article = article.replace(tag,'')
            if len(tags) == 0:
                #print('[[File:')
                break

        # filter [[Image: tags
        while True:
            wikicode = hell.parse(article)
            tags = [str(j) for j in wikicode.filter() if '[image:' in str(j).replace(' ','').lower()]
            for tag in tags:
                article = article.replace(tag,'')
            if len(tags) == 0:
                #print('[[Image:')
                break
                
        # filter [[Category: tags
        while True:
            wikicode = hell.parse(article)
            tags = [str(j) for j in wikicode.filter() if '[category:' in str(j).replace(' ','').lower()]
            for tag in tags:
                article = article.replace(tag,'')
            if len(tags) == 0:
                #print('[[Image:')
                break

        # convert [[link|word]] to word:
        while True:
            wikicode = hell.parse(article)
            tags = [str(j) for j in wikicode.filter() if '[[' in str(j) and len(str(j).split('|')) == 2]
            new_tags = [str(j).replace('[','').replace(']','').split('|')[-1] for j in tags]
            for j in range(len(tags)):
                article = article.replace(tags[j], new_tags[j])
            if len(tags) == 0:
                #print('double_word')
                break
                
        # filter <gallery> tags
        while True:
            wikicode = hell.parse(article)
            tags = [str(j) for j in wikicode.filter_tags() if '<gall' in str(j)]
            for tag in tags:
                article = article.replace(tag, '')
            if len(tags) == 0:
                break
                
        # filter <span> tags
        while True:
            wikicode = hell.parse(article)
            tags = [str(j) for j in wikicode.filter_tags() if '<spa' in str(j)]
            for tag in tags:
                article = article.replace(tag, '')
            if len(tags) == 0:
                break
                
        # filter remaining [[tags]]
        while True:
            wikicode = hell.parse(article)
            tags = [str(j) for j in wikicode.filter() if '[[' in str(j) and '<math' not in str(j)]
            new_tags = [j.replace('[','').replace(']','') for j in tags]
            for j in range(len(tags)):
                article = article.replace(tags[j], new_tags[j])
            if len(tags) == 0:
                break

        new_articles.append(article)


    # check that everything has been filtered properly    
    articles = new_articles

    html_text = []
    comments = []
    arguments = []
    templates = []
    links = []
    tables = []
    files = []
    images = []
    categories = []
    double_words = []
    galleries = []
    spans = []
    remaining_tags = []

    for i in range(len(articles)):
        wikicode = hell.parse(articles[i])
        
        html_text.extend([str(j) for j in wikicode.filter_html_entities()])
        comments.extend(wikicode.filter_comments())
        arguments.extend(wikicode.filter_arguments())
        templates.extend(get_templates(wikicode))
        links.extend(wikicode.filter_external_links())
        tables.extend([str(j) for j in wikicode.filter() if 'class=' in str(j).replace(' ','')])
        files.extend([str(j) for j in wikicode.filter() if '[File:' in str(j).replace(' ','')])
        images.extend([str(j) for j in wikicode.filter() if '[Image:' in str(j).replace(' ','')])
        categories.extend([str(j) for j in wikicode.filter() if '[Category:' in str(j).replace(' ','')])
        double_words.extend([str(j) for j in wikicode.filter() if '[[' in str(j) and len(str(j).split('|')) == 2])
        galleries.extend([str(j) for j in wikicode.filter_tags() if '<gall' in str(j)])
        spans.extend([str(j) for j in wikicode.filter_tags() if '<spa' in str(j)])
        remaining_tags.extend([str(j) for j in wikicode.filter() if '[[' in str(j) and '<math' not in str(j)])

        tags = list(dict.fromkeys([str(i) for i in wikicode.filter_tags()]))
        tag_count = 0
        for tag in tags:
            if '<math' not in tag:
                tag_count += tag.count("'''") + tag.count("''")
    
    values = [len(html_text), len(comments), len(arguments), len(templates), len(links), tag_count, len(tables), len(files), len(images), len(categories), len(double_words), len(galleries), len(spans), len(remaining_tags)]
    
    if list(dict.fromkeys(values)) == [0]:
        break
    else:
        print(values)

  0%|          | 0/8620 [00:00<?, ?it/s]

[0, 0, 0, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


  0%|          | 0/8620 [00:00<?, ?it/s]

In [20]:
# use headings as a check for which articles are too short or unsuitable 

error_headings = []
final_headings = []
for i in range(len(articles)):
    #print(i)
    article = articles[i]
    wikicode = hell.parse(article)
    try:
        final_headings.append((get_headings(wikicode, check=True)[-1], i))
    except:
        error_headings.append(i)
        
# look through final_headings for correct entries, then error_headings to exclude bad articles

In [22]:
len(error_headings)

924

In [23]:
# find bad_articles

i = 0
article = articles[error_headings[i]]
print(error_headings[i])
wikicode = hell.parse(article)

wikicode.filter_headings()

print(article)

148





In computing, floating-point arithmetic (FP) is arithmetic using formulaic representation of real numbers as an approximation to support a trade-off between range and precision. For this reason, floating-point computation is often used in systems with very small and very large real numbers that require fast processing times. In general, a floating-point number is represented approximately with a fixed number of significant digits (the significand) and scaled using an exponent in some fixed base; the base for the scaling is normally two, ten, or sixteen. A number that can be represented exactly is of the following form:
: <math>\text{significand} \times \text{base}^\text{exponent},</math>
where significand is an integer, base is an integer greater than or equal to two, and exponent is also an integer.
For example:
: <math>1.2345 = \underbrace{12345}_\text{significand} \times \underbrace{10}_\text{base}\!\!\!\!\!\!^{\overbrace{-4}^\text{exponent}}</math>.

The term floating poin

In [24]:
# remake articles csv with category data
#bad_articles = [179, 3279, 3293, 3362] # found by checking through error headings using above box
bad_articles = error_headings

categories, titles = list(df.Categories), list(df['Title'])

data = [[articles[i], categories[i], titles[i]] for i in range(len(articles)) if i not in bad_articles]

df = pd.DataFrame(data,columns=['article','categories','title'])

df.to_csv('math_articles.csv',encoding='UTF-8',index=None)

**Can start from here**

In [10]:
df = pd.read_csv('math_articles.csv',encoding='UTF-8')
articles = list(df.article)
df

,article,categories,title
0,\n\n\nAnalysis of variance (ANOVA) is a collec...,"['Land surface effects on climate', 'Climate c...",Albedo
1,"\n\n\n\n\nIn mathematics, the absolute value o...","['Aristotle', 'Aristotelianism *', '384 BC bir...",Aristotle
2,\n\nAlgorithms for calculating variance play a...,['Means'],Arithmetic mean
3,\n\n\n\n\n\nAugustin-Jean Fresnel ( or ; ; 10 ...,"['Analysis of variance', 'Design of experiment...",Analysis of variance
4,"\n\n\n\n\nIn Euclidean geometry, an angle is t...",['Propositional fallacies'],Affirming the consequent
...,...,...,...
7691,\nN<math>-</math>ary TOPSIS is multi-criteria ...,"['Prismatoid polyhedra', 'Johnson solids']",Pentagonal cupola
7692,Gamma ray cross section - a measure of the pro...,"['Zeta and L-functions', 'Algebraic number the...",Dedekind zeta function
7693,Market equilibrium computation (also called co...,"['Elliptic curves', 'Abelian varieties', 'Pair...",Weil pairing
7694,"In statistics, multinomial regression is a ext...",['Neutron Flux'],Neutron flux


In [11]:
# remove \xa0
articles = [i.replace('\xa0',' ') for i in articles]

In [12]:
# replace Math> with math>
articles = [i.replace('Math>','math>').replace('matH>','math>').replace('MATH>','math>') for i in articles]

In [13]:
# replace <math display='block'> type tags with <math>

new_articles = []
for article in articles:
    inside_math_tag = [i.split('>')[0] for i in article.split('<math')[1:] if i != '']
    for i in inside_math_tag:
        article = article.replace(i, '')
    new_articles.append(article)
articles = new_articles

In [14]:
# replace math with a unique identifier per article

math_ids = []
new_articles = []
loop = tqdm(range(len(articles)))
for i in loop:
    article = articles[i]
    article_math = extract_math(article, just_equations=False)
    for j in range(len(article_math)):
        math = article_math[j]
        article = article.replace(math, 'MATH-%s-%s-MATH'%(str(i),str(j)))
        math_ids.append([math, 'MATH-%s-%s-MATH'%(str(i),str(j))])
    new_articles.append(article)
articles = new_articles

  0%|          | 0/7696 [00:00<?, ?it/s]

In [15]:
#articles = [i.replace('\n',' ') for i in articles]

In [16]:
# extract sections

all_sections = []
for i in range(len(articles)):
    article = articles[i]
    wikicode = hell.parse(article)

    headings = [str(j) for j in get_headings(wikicode, check=False)]

    cutoff_list = ['Seealso',
                   'References',
                   'Footnotes',
                   'Notes',
                   'Bibliography',
                   'Publications',
                   'Footnotesandreferences',
                   'Primarysources',
                   'Citations',
                   'Notesandreferences',
                   'Externallinks',
                   'Furtherreading',
                   'Referencesandfurtherreading',
                   'References/sources',
                   'Importantpublications',
                   'Sources']

    sections = [article.split(headings[0])[0]] + [article.split(headings[j])[1].split(headings[j+1])[0] for j in range(len(headings) - 1)]

    if headings[-1].replace(' ','').replace('=','') not in cutoff_list:
         sections += [article.split(headings[-1])[-1]]
        
    all_sections.append(sections)
    
sections = all_sections

In [17]:
structured_sentences = []

times = []
loop = tqdm(range(len(sections)))
for i in loop:
    
    article_sections = sections[i]
    section_sentences = []
    for j in range(len(article_sections)):
        section = article_sections[j]
        section_segments = [i for i in section.split('\n\n') if i.replace(' ','') != '']
        for segment in section_segments:
            sentences = [[str(sent),j,i] for sent in extract_sentences(segment)] # the actual sentence extraction
            section_sentences.append(sentences)
    structured_sentences.append(section_sentences)

  0%|          | 0/7696 [00:00<?, ?it/s]

C:\Users\JCMea\anaconda3\lib\site-packages\torch\amp\autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
Token indices sequence length is longer than the specified maximum sequence length for this model (907 > 512). Running this sequence through the model will result in indexing errors


In [18]:
data = []
for article in structured_sentences:
    for section in article:
        for sentence in section:
            data.append(sentence)

In [323]:
pd.DataFrame(data, columns=['sentence','section','article']).to_csv('math_sentences.csv',encoding='UTF-8',index=None)
df = pd.read_csv('math_sentences.csv',encoding='UTF-8')
df

,sentence,section,article
0,Analysis of variance (ANOVA) is a collection o...,0,0
1,ANOVA was developed by the statistician Ronald...,0,0
2,"ANOVA is based on the law of total variance, w...",0,0
3,"In its simplest form, ANOVA provides a statist...",0,0
4,While the analysis of variance reached fruitio...,1,0
...,...,...,...
778693,"So, materials can be divided into magnetically...",20,7695
778694,In these kind of materials magnetic and elasti...,20,7695
778695,These effects are dependent on the Weiss domai...,20,7695
778696,When a magnetoelastic material is put under st...,20,7695


In [324]:
sentences = list(df.sentence)
len(sentences)

778698

In [325]:
# randomly sample 100 sentences and check for errors

In [326]:
import random
sample = sentences.copy()
random.shuffle(sample)
sample = sample[:100]

In [327]:
# remove all '\n'

df.sentence = [i.replace('\n',' ') for i in sentences]
sentences = list(df.sentence)

In [328]:
# fix issues with spaces

def fix_spaces_at_end_and_beginning(sentence):
    while True:
        if sentence[0] == ' ':
            sentence = sentence[1:]
        if sentence[-1] == ' ':
            sentence = sentence[:-1]
        if sentence[0] != ' ' and sentence[-1] != ' ':
            break
    return ' '.join([i for i in sentence.split(' ') if i != ''])

df.sentence = [fix_spaces_at_end_and_beginning(i) for i in sentences]

sentences = list(df.sentence)

In [329]:
# fix issues with :MATH
df.sentence = [i.replace(":MATH",'MATH').replace('MATH','math') for i in sentences]
sentences = list(df.sentence)

In [330]:
import string
string.ascii_uppercase

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [331]:
def bad_sentence_check(sentence):
    s = sentence
    if (s[0] == '-') or (s[0] not in string.ascii_uppercase and s[0] != '"') or\
       (s[0] not in string.ascii_uppercase and len(s) < 10):
        return True
    else:
        return False

In [332]:
# create structure where structured_sentence[i][j][k] == sentence

article_nums = list(dict.fromkeys(list(df.article)))
structured_sentences = []
loop = tqdm(article_nums)
for i in loop:
    section_nums = list(dict.fromkeys(list(df[(df.article == i)].section)))
    sections = []
    for j in section_nums:
        section = list(df[(df.article == i) & (df.section == j)].sentence)
        sections.append(section)
    structured_sentences.append(sections)

  0%|          | 0/7696 [00:00<?, ?it/s]

In [333]:
# make a function to add first to second

def fix_sentences_in_section(section):
    # assumes first sentence is good
    # looks for second half of sentences to add onto first half
    new_section = [section[0]]
    for i in range(1, len(section)):
        if bad_sentence_check(section[i]) == False:
            new_section.append(section[i])
        else:
            new_section[-1] = new_section[-1] + ' ' + section[i]
    return new_section

In [335]:
new_structured_sentences = []
for i in range(len(structured_sentences)):
    article = structured_sentences[i]
    new_article = []
    for section in article:
        new_article.append(fix_sentences_in_section(section))
    new_structured_sentences.append(new_article)
structured_sentences = new_structured_sentences

In [336]:
while True:
    count = 0
    new_structured_sentences = []
    loop = tqdm(range(len(structured_sentences)))
    for i in loop:

        sections = []
        for j in range(len(structured_sentences[i])):

            section = structured_sentences[i][j]

            # fix *individual* sentences here
            new_section = []
            for k in range(len(section)):
                sentence = ' '.join([i for i in section[k].split(' ') if i != '']) # no double spaces and space starts

                for swap in [['(e.g.,)',''],
                            ['(i.e.,)',''],
                            ['()',''], 
                            [',.',''],
                            ['.,',''],
                            ['(,','('],
                            [':<','<'],
                            [': :',': '],
                            ['. </math>','.</math>'],
                            [', </math>','</math>,'],
                            [',</math>','</math>,'],
                            ["\</math>",'</math>'],
                            ["\ </math>",'</math>'],
                            ["/math>",'</math>'],
                            ['. :;','. '],
                            ['. :','. '],
                            ['. ;','. '],
                            ['.\n','. '],
                            ['. \n','. '],
                            [',,',','],
                            ['<<','<'],
                            ['< <','<'],
                            ['\n*',','],
                            ['(;','('],
                            ['( ;','('],
                            ['math >','math>'],
                            ['< /math','</math']]:
                    

                    sentence = correct_math_tags(sentence.replace(swap[0],swap[1]))
                    
                    if sentence.replace(' ','')[:2] == ']]':
                        sentence = sentence.replace(']]','')
                        
                    if sentence.replace(' ','')[-2:] == ']]':
                        sentence = ''
                        
                    if sentence.split('<ref')[1:] != []:
                        for s in sentence.split('<ref')[1:]:
                            ref = '<ref'+s.split('/ref>')[0]+'/ref>'
                            sentence = sentence.replace(ref,'')
                            
                    if sentence.split('<div')[1:] != []:
                        for s in sentence.split('<div')[1:]:
                            div = '<div'+s.split('>')[0]+'>'
                            sentence = sentence.replace(div,'')
                            
                    if sentence.split('<small')[1:] != []:
                        for s in sentence.split('<small')[1:]:
                            small_text = '<small'+s.split('/small>')[0]+'/small>'
                            sentence = sentence.replace(small_text,'')
                        
                    if sentence.replace('\n','').replace(' ','')[-1:] == ']':
                        sentence = ''
                        
                        
                new_section.append(sentence) if len(sentence.replace(' ','')) != 0 else 0

            sections.append(new_section)

        new_structured_sentences.append(sections)

    structured_sentences = new_structured_sentences
    print(count)
    if count == 0:
        break

  0%|          | 0/7696 [00:00<?, ?it/s]

0


In [340]:
data = []
for i in range(len(structured_sentences)):
    article = structured_sentences[i]
    for j in range(len(article)):
        section = article[j]
        for k in range(len(section)):
            sentence = section[k]
            data.append([sentence,j,i])
            
df_new = pd.DataFrame(data, columns=['sentence','section','article'])
df_new.to_csv('math_sentences.csv',encoding='UTF-8',index=None)
df = df_new
df

,sentence,section,article
0,Analysis of variance (ANOVA) is a collection o...,0,0
1,ANOVA was developed by the statistician Ronald...,0,0
2,"ANOVA is based on the law of total variance, w...",0,0
3,"In its simplest form, ANOVA provides a statist...",0,0
4,While the analysis of variance reached fruitio...,1,0
...,...,...,...
595184,"So, materials can be divided into magnetically...",20,7695
595185,In these kind of materials magnetic and elasti...,20,7695
595186,These effects are dependent on the Weiss domai...,20,7695
595187,When a magnetoelastic material is put under st...,20,7695


In [341]:
df = pd.read_csv('math_sentences.csv',encoding='UTF-8')
df

,sentence,section,article
0,Analysis of variance (ANOVA) is a collection o...,0,0
1,ANOVA was developed by the statistician Ronald...,0,0
2,"ANOVA is based on the law of total variance, w...",0,0
3,"In its simplest form, ANOVA provides a statist...",0,0
4,While the analysis of variance reached fruitio...,1,0
...,...,...,...
595184,"So, materials can be divided into magnetically...",20,7695
595185,In these kind of materials magnetic and elasti...,20,7695
595186,These effects are dependent on the Weiss domai...,20,7695
595187,When a magnetoelastic material is put under st...,20,7695


In [374]:
df = pd.read_csv('math_sentences.csv',encoding='UTF-8')

df = df[df['sentence'].apply(lambda x: len([i for i in x.replace(',','').replace(' ','') if i != '']) > 9)]

df.index = range(len(df))

df.to_csv('math_sentences.csv',encoding='UTF-8',index=None)
df

,sentence,section,article
0,Analysis of variance (ANOVA) is a collection o...,0,0
1,ANOVA was developed by the statistician Ronald...,0,0
2,"ANOVA is based on the law of total variance, w...",0,0
3,"In its simplest form, ANOVA provides a statist...",0,0
4,While the analysis of variance reached fruitio...,1,0
...,...,...,...
593693,"So, materials can be divided into magnetically...",20,7695
593694,In these kind of materials magnetic and elasti...,20,7695
593695,These effects are dependent on the Weiss domai...,20,7695
593696,When a magnetoelastic material is put under st...,20,7695


In [376]:
sentence = list(df.sentence)[1000]
sentence

'He supported that hypothesis by supposing that if the density of the external aether was taken as unity, the density of the internal aether was math-3-33-math, of which the excess, namely math-3-34-math, was dragged along at velocity math-3-31-math, whence the average velocity of the internal aether was math-3-32-math.'

In [408]:
# put math back in sentences

def put_back_math_in_sentence(sentence):
    for math, ID in math_ids:
        sentence = sentence.replace(ID,math).replace('\,','').replace('\;','')
    return sentence

In [410]:
sentences = list(df.sentence)

new_sentences = []
loop = tqdm(sentences)
for sentence in loop:
    if ('math-' in sentence) and ('-math' in sentence):
        new_sentences.append(put_back_math_in_sentence(sentence))
    else:
        new_sentences.append(sentence)
    
df.sentence = new_sentences

df.to_csv('math_sentences.csv',encoding='UTF-8',index=None)
df

  0%|          | 0/593698 [00:00<?, ?it/s]

,sentence,section,article
0,Analysis of variance (ANOVA) is a collection o...,0,0
1,ANOVA was developed by the statistician Ronald...,0,0
2,"ANOVA is based on the law of total variance, w...",0,0
3,"In its simplest form, ANOVA provides a statist...",0,0
4,While the analysis of variance reached fruitio...,1,0
...,...,...,...
593693,"So, materials can be divided into magnetically...",20,7695
593694,In these kind of materials magnetic and elasti...,20,7695
593695,These effects are dependent on the Weiss domai...,20,7695
593696,When a magnetoelastic material is put under st...,20,7695


In [411]:
# check that there is no math left

In [ ]:
df = pd.read_csv('math_sentences.csv',encoding='UTF-8')
df

In [431]:
sentences = list(df.sentence)

In [458]:
# create structure where structured_sentence[i][j][k] == sentence

article_nums = list(dict.fromkeys(list(df.article)))
structured_sentences = []
loop = tqdm(article_nums)
for i in loop:
    section_nums = list(dict.fromkeys(list(df[(df.article == i)].section)))
    sections = []
    for j in section_nums:
        section = list(df[(df.article == i) & (df.section == j)].sentence)
        sections.append(section)
    structured_sentences.append(sections)

  0%|          | 0/7696 [00:00<?, ?it/s]

In [433]:
# new
eq_count = 0
math_count = 0
qid_count = 0
disp_count = 0

for sentence in sentences:
    for j in sentence.split('<math')[1:]:
        math = j.split('</math>')[0]
        
        if 'display' in math:
            disp_count += 1
        
        if 'qid' in math:
            qid_count += 1
            
    eq_count += len(extract_math(sentence, just_equations=True))
    math_count += len(extract_math(sentence, just_equations=False))
    

eq_count, math_count, qid_count, disp_count

(193104, 473368, 0, 2300)

**Binary Sentence Ordering**

In [459]:
data = []

for article in structured_sentences:
    for section in article:
        for i in range(len(section)-1):
            first = section[i]
            second = section[i+1]
    
            value = np.random.choice(range(2))
            if value == 0:
                out = [0, first, second]
            else:
                out = [1, second, first]
            data.append(out)
            
df_bso = pd.DataFrame(data,columns=['value','S0','S1'])
df_bso.to_csv('BSO_data_math.csv',encoding='UTF-8',index=None)
len(df_bso)

514057

In [460]:
# check

i = 100038

for part in data[i]:
    print(part,'\n\n','-'*75,'\n')
    
len([i[0] for i in data if i[0] == 0])/len(data)

0 

 --------------------------------------------------------------------------- 

One is that the smooth structure allows differential equations to be posed on the manifold; the tangent and cotangent spaces provide enough structure to perform calculus on manifolds. 

 --------------------------------------------------------------------------- 

Of key interest is the Laplacian, and, with a constant term, what amounts to the Klein–Gordon operator. 

 --------------------------------------------------------------------------- 



0.5001488161818631

**Sentence Position**

In [44]:
data = []

for article in structured_sentences:
    for section in article:
        if len(section) > 4:
            cluster = section[:5]
            value = np.random.choice(range(5))
            sentence = cluster[value]
            
            cluster.remove(sentence)
            cluster.insert(0,sentence)
            
            data.append([value] + cluster)
            
df_sp = pd.DataFrame(data,columns=['value','S0','S1','S2','S3','S4'])
df_sp.to_csv('SP_data_math.csv',encoding='UTF-8',index=None)
len(df_sp)

48088

**Discourse Coherence**

In [45]:
data = []
# random sentence sampled from different wikipedia article
for i in range(len(structured_sentences)):
    article = structured_sentences[i]
    for j in range(len(article)):
        section = article[j]
        
        if len(section) > 5:
            cluster = section[:6]
            index = np.random.choice(range(1,5))
            sentence = cluster[index]
            
            value = np.random.choice(range(2))
            if value == 1:
                while True:
                    rand_art_idx = np.random.choice(len(structured_sentences))
                    if rand_art_idx != i:
                        break

                rand_article = structured_sentences[rand_art_idx]
                rand_section = rand_article[np.random.choice(len(rand_article))]
                random_sentence = rand_section[np.random.choice(len(rand_section))]

                cluster.remove(sentence)
                cluster.insert(index, random_sentence)
            
            out = [value] + cluster
            
            data.append(out)
                
df_dc = pd.DataFrame(data,columns=['value','S0','S1','S2','S3','S4','S5'])
df_dc.to_csv('DC_data_math.csv',encoding='UTF-8',index=None)
len(df_dc)

41020

**Sentence Section Prediction**

In [23]:
intro_sents = []
not_intro_sents = []

for article in structured_sentences:
    intro = article[0]
    for sentence in intro:
        intro_sents.append([1, sentence])
        

while True:
    article = structured_sentences[rand_art_idx]
    section = article[np.random.choice(len(article))]
    random_sentence = section[np.random.choice(len(section))]
    not_intro_sents.append([0,random_sentence])
    if len(not_intro_sents) == len(intro_sents):
        break
        
data = intro_sents + not_intro_sents
#np.random.shuffle(data)

df_ssp = pd.DataFrame(data,columns=['value','S0'])
df_ssp.to_csv('SSP_data.csv',encoding='UTF-8',index=None)
len(df_ssp)

90714